In [1]:
# Install necessary packages
!pip install chembl_webresource_client pandas scikit-learn seaborn tensorflow torch rdkit xgboost scikit-optimize


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from xgboost import XGBRegressor
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args


In [17]:
# Select the best target (Cytochrome P450 1A2)
selected_target = "CHEMBL3356"

# Retrieve only IC50 data
result = new_client.activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

# Convert to DataFrame
df = pd.DataFrame.from_dict(result)

# Save the dataset
df.to_csv("best_target_bioactivity.csv", index=False)

# Print the first few rows
print(f"Retrieved {df.shape[0]} IC50 values for target {selected_target}")
df.head()


Retrieved 5279 IC50 values for target CHEMBL3356


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,31874,[],CHEMBL666153,Inhibition of cytochrome P450 1A2 of isolated ...,A,None,None,BAO_0000190,...,Homo sapiens,Cytochrome P450 1A2,9606,None,None,IC50,uM,UO_0000065,None,6.0
1,None,None,45774,[],CHEMBL666250,Inhibition of human cytochrome P450 1A2 activi...,A,None,None,BAO_0000190,...,Homo sapiens,Cytochrome P450 1A2,9606,None,None,IC50,uM,UO_0000065,None,10.0
2,None,None,60243,[],CHEMBL666153,Inhibition of cytochrome P450 1A2 of isolated ...,A,None,None,BAO_0000190,...,Homo sapiens,Cytochrome P450 1A2,9606,None,None,IC50,uM,UO_0000065,None,31.0
3,None,None,61665,[],CHEMBL666153,Inhibition of cytochrome P450 1A2 of isolated ...,A,None,None,BAO_0000190,...,Homo sapiens,Cytochrome P450 1A2,9606,None,None,IC50,uM,UO_0000065,None,100.0
4,None,None,62458,[],CHEMBL666250,Inhibition of human cytochrome P450 1A2 activi...,A,None,None,BAO_0000190,...,Homo sapiens,Cytochrome P450 1A2,9606,None,None,IC50,uM,UO_0000065,None,66.0


In [18]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors

# Load the dataset
df = pd.read_csv("best_target_bioactivity.csv")

# Drop missing values
df = df[df['standard_value'].notna()]
df = df[df['canonical_smiles'].notna()]

# Convert IC50 values to numeric and filter out negative values (invalid)
df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')
df = df[df['standard_value'] > 0]  # Remove invalid IC50 values

# Convert IC50 (µM) to pIC50
df['pIC50'] = -np.log10(df['standard_value'] * 1e-6)

# Drop unnecessary columns
df = df[['canonical_smiles', 'pIC50']]

# Save cleaned dataset
df.to_csv("cleaned_bioactivity_data.csv", index=False)

print(f"✅ Cleaned dataset saved! {df.shape[0]} molecules available for modeling.")
df.head()


✅ Cleaned dataset saved! 4239 molecules available for modeling.


,canonical_smiles,pIC50
0,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccc...,2.221849
1,COc1ccc(NS(=O)(=O)c2ccc(Br)cc2)cc1N1CCN(C)CC1,2.000000
2,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4ccc(-c5cccc...,1.508638
3,CCn1c2ccccc2c2cc(NC(=O)CN3CCN(C[C@@H](O)COc4cc...,1.000000
4,COc1ccc(NS(=O)(=O)c2sc3ccc(Cl)cc3c2C)cc1N1CCNC...,1.180456


In [19]:
from rdkit.Chem import rdMolDescriptors

# Function to compute molecular descriptors
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return [
            Descriptors.MolWt(mol),  
            Descriptors.MolLogP(mol),  
            Descriptors.NumHDonors(mol),  
            Descriptors.NumHAcceptors(mol),
            rdMolDescriptors.CalcTPSA(mol),  # Topological Polar Surface Area
            Descriptors.NumRotatableBonds(mol)  # Molecular Flexibility
        ]
    else:
        return [None] * 6

# Apply feature extraction
df[['MolWt', 'LogP', 'HDonors', 'HAcceptors', 'TPSA', 'RotatableBonds']] = df['canonical_smiles'].apply(lambda x: pd.Series(calculate_descriptors(x)))

# Drop missing descriptor values
df.dropna(inplace=True)

# Save enhanced dataset
df.to_csv("molecular_descriptors.csv", index=False)

print(f"✅ Molecular descriptors extracted! {df.shape[0]} molecules available for modeling.")
df.head()


✅ Molecular descriptors extracted! 4239 molecules available for modeling.


,canonical_smiles,pIC50,MolWt,LogP,HDonors,HAcceptors,TPSA,RotatableBonds
0,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccc...,2.221849,516.667,4.26772,2.0,7.0,77.93,9.0
1,COc1ccc(NS(=O)(=O)c2ccc(Br)cc2)cc1N1CCN(C)CC1,2.000000,440.363,3.01030,1.0,5.0,61.88,5.0
2,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4ccc(-c5cccc...,1.508638,516.667,4.26772,2.0,7.0,77.93,9.0
3,CCn1c2ccccc2c2cc(NC(=O)CN3CCN(C[C@@H](O)COc4cc...,1.000000,557.720,4.72852,2.0,8.0,82.86,9.0
4,COc1ccc(NS(=O)(=O)c2sc3ccc(Cl)cc3c2C)cc1N1CCNC...,1.180456,488.462,4.50392,2.0,6.0,70.67,5.0


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
df = pd.read_csv("molecular_descriptors.csv")

# Define features & target
X = df.drop(columns=['pIC50', 'canonical_smiles'])
y = df['pIC50']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Train XGBoost
xgb_model = XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate models
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"📊 Model Performance:")
print(f"✅ Random Forest: RMSE = {rmse_rf}, R² = {r2_rf}")
print(f"✅ XGBoost: RMSE = {rmse_xgb}, R² = {r2_xgb}")


📊 Model Performance:
✅ Random Forest: RMSE = 0.7201988536486071, R² = 0.4033977066215242
✅ XGBoost: RMSE = 0.7297281749747517, R² = 0.38750535430905697


In [22]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
df = pd.read_csv("molecular_descriptors.csv")

# Define features & target
X = df.drop(columns=['pIC50', 'canonical_smiles'])
y = df['pIC50']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameter tuning for XGBoost
space = [
    Integer(50, 500, name="n_estimators"),
    Real(0.01, 0.3, name="learning_rate"),
    Integer(3, 10, name="max_depth")
]

def objective(params):
    n_estimators, learning_rate, max_depth = params
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))

# Run Bayesian Optimization
res = gp_minimize(objective, space, n_calls=20, random_state=42)

# Train optimized models
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

xgb_model = XGBRegressor(n_estimators=res.x[0], learning_rate=res.x[1], max_depth=res.x[2], random_state=42)
xgb_model.fit(X_train, y_train)

# Ensemble Model (Averaging)
def ensemble_predict(X):
    rf_pred = rf_model.predict(X)
    xgb_pred = xgb_model.predict(X)
    return (rf_pred + xgb_pred) / 2  # Simple averaging

# Predictions
y_pred_ensemble = ensemble_predict(X_test)

# Evaluate performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
r2 = r2_score(y_test, y_pred_ensemble)

print(f"📊 Final Model Performance: RMSE = {rmse}, R² = {r2}")

# Save models and scaler
joblib.dump(rf_model, "random_forest_model.pkl")
joblib.dump(xgb_model, "xgboost_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Optimized models and scaler saved successfully!")


📊 Final Model Performance: RMSE = 0.7097266700590115, R² = 0.42062157636710396
✅ Optimized models and scaler saved successfully!
